## 一、PyTorch基础知识

#### 1.1、PyTorch的诞生

2017年1月，Faceboo人工智能研究院团队在GitHub上开源了PyTorch框架，PyTorch的历史可追溯到2002年诞生的Torch。Torch使用了一种不是很大众的语言Lua作为接口。考虑到Python在计算科学领域的领先地位，在2017年Torch的幕后团队推出了PyTorch。PyTorch不是简单地封装Lua Torch提供的Python接口，而是对Tensor之上的所有模块进行了重构，并新增了最先进的自动求导系统

#### 1.2Pytorch的安装

在PyTorch官网提供了pip，conda方式安装
![avatar](2_1.png)

#### 1.3、PyTorch入门

###### 1.3.1、Tensor

   Tensor是PyTorch中重要的数据结构，可认为是一个高维数组。它可以是一个数（标量）、一维数组（向量）、二维数组（矩阵）或更高维的数组。Tensor的使用和numpy的ndarrays类似。

In [2]:
import torch as t

In [2]:
# 构建5*3矩阵，只分配了空间，未初始化
x = t.Tensor(5, 3)
x

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.4742e-42, 0.0000e+00],
        [0.0000e+00, 7.5338e+28, 0.0000e+00]])

In [3]:
# 使用[0, 1]均匀分布随机初始化二维数组
x = t.rand(5, 3)
x

tensor([[0.1716, 0.7072, 0.6712],
        [0.2445, 0.3625, 0.4225],
        [0.4019, 0.1662, 0.7261],
        [0.4422, 0.9970, 0.3754],
        [0.4961, 0.2674, 0.4745]])

In [4]:
# 查看x的形状
print(x.size())
# 查看列的个数，两种写法等价
x.size()[0], x.size(1)  

torch.Size([5, 3])


**注意**，函数名后面带下划线的函数会修改Tensor本身。例如，x.add_(y)和x.t_()会改变x，但x.add(y)和x.t()会返回一个新的Tensor，而x不变。

Tensor还支持很多操作，包括数学运算、线性代数、选择、切片等，其接口设计与numpy极为相似。

###### 1.3.2、Autograd:自动微分

深度学习的算法本质上是通过反向传播求导数，PyTorch的Autograd模块实现了此功能。在Tensor上的所有操作，Autograd都能为它们自动提供微分。

autograd.Variable是Autograd中的核心类，它简单地封装了Tensor，并支持几乎所有的Tensor操作。Tensor在被封装为Variable之后，可以调用它的.backward实现反向传播，自动计算所有梯度。

Variable主要包含三个属性：
* data：保存Variable所包含的Tensor
* grad：保存data对应的梯度，grad也是个Variable，而不死Tensor，它和data的形状一样
* grad_fn：指向一个Function对象，这个Function用来反向传播计算输入的梯度。

In [3]:
from torch.autograd import Variable

In [4]:
# 使用Tensor新建一个Variable
x = Variable(t.ones(2, 2), requires_grad = True)
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [5]:
y = x.sum()
y

tensor(4., grad_fn=<SumBackward0>)

In [6]:
y.grad_fn

In [9]:
# 反向传播，计算梯度
y.backward()
y

tensor(4., grad_fn=<SumBackward0>)

In [8]:
# y = x.sum() = (x[0][0] + x[0][1] + x[1][0] + x[1][1])
# 每个值的梯度都为1
x.grad

tensor([[1., 1.],
        [1., 1.]])

**注意**：grad在反向传播过程中是累加的，这意味着每次运行反向传播，梯度都会累加之前的梯度，所以反向传播之前需把梯度清零

In [11]:
y.backward()
x.grad

tensor([[4., 4.],
        [4., 4.]])

In [12]:
y.backward()
x.grad

tensor([[5., 5.],
        [5., 5.]])

In [13]:
# 对x清理
x.grad.data.zero_()

tensor([[0., 0.],
        [0., 0.]])

In [14]:
y.backward()
x.grad

tensor([[1., 1.],
        [1., 1.]])

###### 1.3.3、神经网络

Autograd实现了反向传播功能，但是直接用来写深度学习的代码在很多情况下还是稍显复杂，torch.nn是专门为神经网络设计的模块化接口。nn构建于Autograd之上，可用来定义和运行神经网络。nn.Moudle是nn中最重要的类，可以把它看作一个网络的封装，包含网络各层定义及forward方法。forward方法可以返回前向传播的结果。

**定义网络**
定义网络时，需要继承nn.Moudle，并实现它的forward方法，把网络中具有学习参数的层放在构造函数__init__中。如果某一层（如ReLU）不具有可学习的参数，则既可以放在构造函数中，也可以不放。

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        # nn.Moudle子类的函数必须在构造函数中执行父类的构造函数
        super(Net, self).__init__() # 等价于nn.Module.__init__(self)  
        self.conv1 = nn.Conv2d(1, 6, 5)  # 1表示输入图片为单通道，6表示输出通道数，5表示卷积核为5*5
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 仿射层/全连接层，y=Wx+b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # 卷积 -> 激活 -> 池化
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # reshape, '-1'表示自适应
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


只要在nn.Module的子类中定义了forward函数，backward函数就会被自动实现（利用Autograd）。在forward函数中可使用任何Variable支持的函数，还可以使用if、for循环、print、log等Python语法，写法和标准的Python写法一致。

In [24]:
params = list(net.parameters())
print(len(params))

10


In [25]:
for name, parameters in net.named_parameters():
    print(name, ':', parameters.size())

conv1.weight : torch.Size([6, 1, 5, 5])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 5, 5])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 400])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])


forward函数的输入和输出都是Variable，只有Variable才具有自动求导功能，Tensor是没有的，所以在输入时，需要把Tensor封装成Variable。

In [39]:
input = t.randn(1, 1, 32, 32)
out = net(input)
out.size()

torch.Size([1, 10])

In [40]:
net.zero_grad() # 所有参数的梯度清零
out.backward(Variable(t.ones(1, 10))) # 反向传播

**损失函数**   nn实现了神经网络中大多数的损失函数，例如nn.MSELoss用来计算均方误差，nn.CrossEntropyLoss用来计算交叉熵损失

In [45]:
output = net(input)
target = t.arange(0,10).view(1,10) 
criterion = nn.MSELoss()
loss = criterion(output, target)
loss # loss是个scala

RuntimeError: Expected object of scalar type Float but got scalar type Long for argument #2 'target'

如果对loss进行反向传播溯源(使用gradfn属性)，可看到它的计算图如下：

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d  
      -> view -> linear -> relu -> linear -> relu -> linear 
      -> MSELoss
      -> loss
      
当调用loss.backward()时，该图会动态生成并自动微分，也即会自动计算图中参数(Parameter)的导数。

In [44]:
# 运行.backward，观察调用之前和调用之后的grad
net.zero_grad() # 把net中所有可学习参数的梯度清零
print('反向传播之前 conv1.bias的梯度')
print(net.conv1.bias.grad)
loss.backward()
print('反向传播之后 conv1.bias的梯度')
print(net.conv1.bias.grad)

反向传播之前 conv1.bias的梯度
tensor([0., 0., 0., 0., 0., 0.])


NameError: name 'loss' is not defined

**优化器** 在反向传播计算完所有参数的梯度后，还需要使用优化方法来更新网络的权重和参数，例如随机梯度下降法(SGD)的更新策略如下

weight = weight - learning_rate * gradient

#### 1.4、CIFAR-10分类

实现对CIFAR-10数据集的分类，步骤如下：
1. 使用torchvision加载并预处理CIFAR-10数据集
2. 定义网络
3. 定义损失函数和优化器
4. 训练网络并更新网络参数
5. 测试网络

**CIFAR-10数据加载及预处理**

CIFAR-101是一个常用的彩色图片数据集，它有10个类别: 'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'。
每张图片都是$3\times32\times32$，也即3-通道彩色图片，分辨率为 $ 32\times32 $ 。

In [7]:
import torchvision as tv
import torchvision.transforms as transforms
from torchvision.transforms import ToPILImage
show = ToPILImage()

In [ ]:
# 定义对数据的预处理
transform = transforms.Compose([transforms.ToTensor(),  # 转为Tensor
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  # 归一化

trainset = tv.datasets.CIFAR10(root='data', train=True, download=True, transform=transform)
trainloader = t.utils,data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)














0it [00:00, ?it/s]












  0%|                                                                                    | 0/170498071 [00:18<?, ?it/s]












  0%|                                                                    | 8192/170498071 [00:23<33:32:09, 1412.17it/s]












  0%|                                                                   | 24576/170498071 [00:28<27:41:01, 1710.53it/s]












  0%|                                                                   | 40960/170498071 [00:31<21:32:20, 2198.29it/s]












  0%|                                                                   | 57344/170498071 [00:33<17:05:34, 2769.85it/s]












  0%|                                                                   | 73728/170498071 [00:36<14:15:28, 3320.29it/s]












  0%|                                                                   | 90112/170498071 [00:42<15:16:23, 3099.24it/s]












  0%|                           

In [ ]:
testset = tv.datasets.CIFAR10(root='data', train=False, download=True, transform=transform)
testloader = t.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
(data, label) = trainset[100]
print(classes[label])

show((data+1)/2).resize((100, 100))

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(' '.join('%11s'%classes[labels[j]] for j in range(4)))
show(tv.utils.make_grid((images+1)/2)).resize((400,100))

**定义网络**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() 
        self.conv1 = nn.Conv2d(3, 6, 5) 
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
print(net)

**定义损失函数和优化器**

In [ ]:
from torch import optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.paramaters(), lr=0.001, momentum=0.9)

**训练网络**

所有网络的训练流程都是类似的，不断地执行如下流程：
* 输入数据
* 前向传播+反向传播
* 更新参数

In [ ]:
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 输入数据
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        # 梯度清零
        optimizer.zero_grad()
        # forward+backward
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        # 更新参数
        optimizer.step()
        # 打印log信息
        running_loss += loss.data[0]
        if i % 2000 == 1999: # 每2000个batch打印一下训练状态
            print('[%d, %5d] loss: %.3f' \
                  % (epoch+1, i+1, running_loss / 2000))
            running_loss = 0.0
print('Finished Training')

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next() # 一个batch返回4张图片
print('实际的label: ', ' '.join(\
            '%08s'%classes[labels[j]] for j in range(4)))
show(tv.utils.make_grid(images / 2 - 0.5)).resize((400,100))

In [ ]:
# 计算图片在每个类别上的分数
outputs = net(images)
# 得分最高的那个类
_, predicted = t.max(outputs.data, 1)

print('预测结果: ', ' '.join('%5s'\
            % classes[predicted[j]] for j in range(4)))

In [ ]:
correct = 0 # 预测正确的图片数
total = 0 # 总共的图片数


# 由于测试的时候不需要求导，可以暂时关闭autograd，提高速度，节约内存
with t.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = t.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

print('10000张测试集中的准确率为: %d %%' % (100 * correct / total))

**在GPU上训练**

In [ ]:
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")

net.to(device)
images = images.to(device)
labels = labels.to(device)
output = net(images)
loss= criterion(output,labels)

loss

**感谢**：深度学习框架PyTorch：入门与实践（陈云著）